In [ ]:
# ✅todo : 1. colab 용으로 requirements.txt을 하나 더 만든다.
# ✅ todo : 1. turnonselenium 내용을 colab용으로 추가한다.
# ✅todo : 1. blogger를 모듈화해서 사용하기 쉽게 만든다.
# ✅todo : 1. 쓰레딩 시키는 것도 함수화한다.
# ✅todo : 1. 코랩은 쓰레딩 몇개를 시키는게 적당한지를 확인한다 = 2개 (2코어라서)
# ✅todo : 1. 코랩크롤링 시에, user-agent 사용하도록 한다.
# todo : duckduckgo 관련자료 서칭 대체품 찾기
# todo : 쉬는 시간 가지도록 만들기
# todo : 만들어진 컨텐츠들 중에 document 없는 경우 찾기


# ✅todo : 1딸깍 테스트
# todo : 1딸깍 반복문에 쉬는 시간 부여하기
# todo : 2딸깍 테스트
# todo : 2딸깍 반복문에 쉬는 시간 부여하기
# todo : 3딸깍 테스트
# todo : 3딸깍 반복문에 쉬는 시간 부여하기

# ✅todo : 1딸깍 테스트 - colab
# todo : 2딸깍 테스트 - colab
# todo : 3딸깍 테스트 - colab

# todo : 추가 키워드 수집 작업 시 기존 csv를 덮어쓸지 말지를 결정하도록 해주세요. - 현재는 덮어씌우지 않는 것으로 되어있음.
# todo : 추가 키워드 수집 작업 시 기존 suitable keywords 리스트를 고려하도록 해주세요. (있으면 재반복해서 수행하지는 않는 것으로.)

# todo : 업로드가 성공적으로 수행되면은 옮기는 것으로 변경하세요.


# 기본실행

In [37]:
# 기본적인 환경설정을 합니다.
isHeadless = True
verbose = True


from module.Crawler import Crawler
from module.File_manager import File_manager
from module.Blogger import Blogger
from module.AIAgent import AIAgent
from module.Uploader import Uploader

# 경고를 무시합니다.
import warnings
warnings.filterwarnings("ignore")

# 환경변수 저장 (.env) 확인
from dotenv import load_dotenv
import os

load_dotenv() 


verbose = True
isHeadless = False

# tester용 객체들을 만듭니다.
crawler_tester = Crawler(isHeadless = isHeadless, verbose = verbose)
File_manager_tester = File_manager(blogname='tester', verbose=verbose)
aiagent_tester = AIAgent(blogname='tester', verbose = verbose)
uploader_tester = Uploader(blogname='tester', verbose = verbose)
blogger_tester = Blogger(blogname='tester', verbose = verbose, isHeadless = isHeadless)

 

기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
✅ 기존 벡터스토어 로드되었습니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
✅ 기존 벡터스토어 로드되었습니다.
✅ 기존 벡터스토어 로드되었습니다.
True
True
Data loaded from JSON


In [3]:


# blogger 클래스를 호출합니다.
from module.Blogger import Blogger
blog_names = ['statifi', 'kindmom', 'sweetkiwi', 'takuz', 'morningbbobbo']
bloggers = {}
for blogname in blog_names:
    print(f'🌐 now loading : {blogname}.')
    bloggers[blogname] = Blogger(blogname = blogname, verbose = verbose, isHeadless = isHeadless)

def thread_bloggers(func, bloggers, **kwargs):
    import threading

    threads = []
    for blogger in bloggers.values():
        # args 튜플에 blogger를 포함시키고, kwargs 딕셔너리를 threading.Thread에 전달
        thread = threading.Thread(target=func, args=(blogger,), kwargs=kwargs)
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()


🌐 now loading : statifi.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
✅ 기존 벡터스토어 로드되었습니다.
✅ 기존 벡터스토어 로드되었습니다.
True
True
Data loaded from JSON
🌐 now loading : kindmom.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
✅ 기존 벡터스토어 로드되었습니다.
✅ 기존 벡터스토어 로드되었습니다.
True
True
Data loaded from JSON
🌐 now loading : sweetkiwi.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
✅ 기존 벡터스토어 로드되었습니다.
✅ 기존 벡터스토어 로드되었습니다.
True
True
Data loaded from JSON
🌐 now loading : takuz.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
✅ 기존 벡터스토어 로드되었습니다.
✅ 기존 벡터스토어 로드되었습니다.
True
True
Data loaded from JSON
🌐 now loading : morningbbobbo.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
✅ 기존 벡터스토어 로드되었습니다.
✅ 기존 벡터스토어 로드되었습니다.
True
True
Data loaded from JSON


# 클릭1 : 키워드 수집

In [ ]:
# 환경설정

# 깊이 정의
depth = int(input('키워드를 수집할 깊이를 입력하세요. (최대 3 이상은 비추천) :'))

# 단어 정의
recommended = {
    "건강과 웰빙": ["요가", "마인드풀니스", "웰니스 리트릿", "건강한 간식", "명상", "필라테스", "건강한 식단", "건강 추세", "힐링", "건강 검진"],
    "기업과 기업가": ["기업가 정신", "크라우드펀딩", "비즈니스 모델 혁신", "기술 벤처", "벤처 투자", "스타트업 생태계"],
    "재택 근무 및 자기 계발 블로그": ["재택 근무 팁", "자기계발 책 추천", "온라인 학습 플랫폼", "시간 관리 기술", "생산성 향상", "사이드 허슬", "재택 근무 환경", "프리랜서 가이드", "자기 관리", "모티베이션 증진"],
    "건강 및 웰니스 블로그": ["건강한 식단", "정신 건강", "스트레스 관리", "홈 피트니스", "명상 기법", "웰니스 라이프스타일", "영양 정보", "건강 검진 가이드", "체중 관리", "건강한 습관"],
    
}

subjects_n_words_for_statifi = {}
subjects_n_words_for_kindmom = {}
subjects_n_words_for_sweetkiwi = {}
subjects_n_words_for_takuz = {}
subjects_n_words_for_morningbbobbo = {}

# 각 블로거의 주제 및 키워드 데이터

total_subjects_n_words = {
    'statifi': subjects_n_words_for_statifi,
    'kindmom': subjects_n_words_for_kindmom,
    'sweetkiwi': subjects_n_words_for_sweetkiwi,
    'takuz': subjects_n_words_for_takuz,
    'morningbbobbo': subjects_n_words_for_morningbbobbo
}



# 함수정의
def collect_keywords_for_blogger(blogger, total_subjects_n_words, depth, save):
    subjects_n_words = total_subjects_n_words[blogger.blogname]
    blogger.collect_keywords(subjects_n_words=subjects_n_words, depth=depth, save=save)


# 실행
thread_bloggers(
    func=collect_keywords_for_blogger,
    bloggers=bloggers,
    total_subjects_n_words=total_subjects_n_words,
    depth=depth,
    save=True
)

In [2]:
# 단일 블로거로 테스트

test_blog = 'tester'
total_subjects_n_words = {test_blog : {'파이어족' : ['파이어족']} }
depth = 1

# 함수정의
def collect_keywords_for_blogger(blogger, total_subjects_n_words, depth, save):
    subjects_n_words = total_subjects_n_words[blogger.blogname]
    blogger.collect_keywords(subjects_n_words=subjects_n_words, depth=depth, save=save)


collect_keywords_for_blogger(blogger_tester, total_subjects_n_words, depth, save=True)

vectorstore_stored_keywords = blogger_tester.keyword_ai.vectorstore_list.index.to_list()
if len(vectorstore_stored_keywords) > 0 :
    print('🌐 키워드 수집이 완료되었습니다.')    
    blogger_tester.keyword_ai.vectorstore_clear()

💛 Now  : Subject : (파이어족) Depth (0) collected_keywords : (0)
파이어족💬 daum에서 suggest keyword 확인
파이어족✅ daum에서 suggest keyword 확인
파이어족💬 daum에서 open keyword 확인
파이어족✅ daum에서 open keyword 확인
파이어족💬 google에서 suggest keyword 확인
파이어족✅ google에서 suggest keyword 확인
파이어족💬 google에서 open keyword 확인
파이어족✅ google에서 open keyword 확인
파이어족 뜻💬  다음 광고확인
파이어족 뜻✅ 다음 광고확인 (0)
파이어족 5억💬  다음 광고확인
파이어족 5억✅ 다음 광고확인 (0)
500만 달러💬  다음 광고확인
500만 달러✅ 다음 광고확인 (0)
파이어족 특징💬  다음 광고확인
파이어족 특징✅ 다음 광고확인 (0)
파이어족 자산💬  다음 광고확인
파이어족 자산✅ 다음 광고확인 (0)
파이어족 목표금액💬  다음 광고확인
파이어족 목표금액✅ 다음 광고확인 (0)
파이어족 문제점💬  다음 광고확인
파이어족 문제점✅ 다음 광고확인 (0)
파이어족 장점💬  다음 광고확인
파이어족 장점✅ 다음 광고확인 (0)
파이어족 실패💬  다음 광고확인
파이어족 실패✅ 다음 광고확인 (0)
욜로족💬  다음 광고확인
욜로족✅ 다음 광고확인 (4)
딩크족💬  다음 광고확인
딩크족✅ 다음 광고확인 (0)
파이어족 계산기💬  다음 광고확인
파이어족 계산기✅ 다음 광고확인 (0)
조기 은퇴💬  다음 광고확인
조기 은퇴✅ 다음 광고확인 (11)
니트족💬  다음 광고확인
니트족✅ 다음 광고확인 (2)
파이어족 은퇴자금💬  다음 광고확인
파이어족 은퇴자금✅ 다음 광고확인 (0)
1000만 달러💬  다음 광고확인
1000만 달러✅ 다음 광고확인 (0)
파이어족 근황💬  다음 광고확인
파이어족 근황✅ 다음 광고확인 (0)
fire💬  다음 광고확인
fire✅ 다음 광고확인 (6)
파이어족 현

# 클릭 2 : 포스팅 생성

In [ ]:
# 환경설정
num_contents_creation = int(input('생성할 컨텐츠 갯수를 입력하세요. 무한대생산은 엔터를 치세요 '))

# 함수정의
def create_contents(blogger, num_contents_creation=5):
    print(f"Starting thread for {blogger.blogname}")
    blogger.create_contents(num_contents_creation)

# 실행
thread_bloggers(create_contents, bloggers, num_contents_creation=num_contents_creation)

In [31]:
# 함수정의
def create_contents(blogger, num_contents_creation=5):
    print(f"Starting thread for {blogger.blogname}")
    blogger.create_contents(num_contents_creation)

# 함수실행

blogger_tester.keyword_ai.vectorstore_save_texts('파이어족 특징') # 예시 키워드 저장
num_of_created_contents_now = len(blogger_tester.file_manager.get_file_names())

try :
    create_contents(blogger_tester, num_contents_creation=1)
finally :
    blogger_tester.keyword_ai.vectorstore_clear()

# 실행결과 확인
num_of_created_contents_after = len(blogger_tester.file_manager.get_file_names())
if (num_of_created_contents_after - num_of_created_contents_now) > 0 :
    print('🌐 컨텐츠 생성이 완료되었습니다.')  


✅ 벡토어스토어 내 새 문서 추가 완료
Starting thread for tester
✅ 벡토어스토어 내 새 문서 추가 완료
파이어족 생태💬 이미지수집
Fire Ecology✅ 이미지수집 (1)
파이어족 행동💬 이미지수집
Fire Behavior✅ 이미지수집 (1)
파이어족 진화💬 이미지수집
Fire Evolution✅ 이미지수집 (1)
contents saved as JSON :  c:\hellodear\tistory_ebook\tester\storage\[[tester]]_((ko))_``파이어족 특징 생태, 행동, 진화에 대해 알아보기``.json
Data successfully saved to c:\hellodear\tistory_ebook\tester\for_upload\[[tester]]_((ko))_``파이어족 특징 생태, 행동, 진화에 대해 알아보기``.txt
🌐 컨텐츠 생성이 완료되었습니다.


# 클릭3 : 티스토리 블로그 업로드

In [ ]:
# 환경설정
num_contents_upload = int(input('업로드 할 컨텐츠 갯수를 입력하세요. 무한대생산은 엔터를 치세요. '))
assert num_contents_upload is not None, '업로드할 컨텐츠 갯수를 입력하세요.'
assert num_contents_upload <= 15 , '15개 이하로 업로드 가능합니다.'

# 함수정의
def upload(blogger, num_contents_upload):
    blogger.upload_contents(num_contents_upload)

# 실행
thread_bloggers(
    func=upload,
    bloggers=bloggers,
    num_contents_upload=num_contents_upload)

테스트코드 (blogger_tester)

In [38]:
# 함수정의
def upload(blogger, num_contents_upload = 1):
    blogger.upload_contents(num_contents_upload)

# 함수실행

os.environ["tester_NEW_POST_URL"] = r"https://statifi.tistory.com/manage/newpost/?type=post&returnURL=%2Fmanage%2Fposts%2F"
os.environ["tester_ID"] = "jsj950611@naver.com"
os.environ["tester_PW"] = "Zhzkzhffk1!"


upload(blogger_tester, num_contents_upload=1)

Data loaded from text file:
업로드 할 문서의 갯수는 1개 입니다.


at alert Message: no such alert
  (Session info: chrome=124.0.6367.119)
Stacktrace:
	GetHandleVerifier [0x00007FF7040E1502+60802]
	(No symbol) [0x00007FF70405AC02]
	(No symbol) [0x00007FF703F17B13]
	(No symbol) [0x00007FF703F08137]
	(No symbol) [0x00007FF703F3B02F]
	(No symbol) [0x00007FF703F8AC2D]
	(No symbol) [0x00007FF703F3AEAB]
	(No symbol) [0x00007FF703F8A923]
	(No symbol) [0x00007FF703F58FEC]
	(No symbol) [0x00007FF703F59C21]
	GetHandleVerifier [0x00007FF7043E411D+3217821]
	GetHandleVerifier [0x00007FF7044260B7+3488055]
	GetHandleVerifier [0x00007FF70441F03F+3459263]
	GetHandleVerifier [0x00007FF70419B846+823494]
	(No symbol) [0x00007FF704065F9F]
	(No symbol) [0x00007FF704060EC4]
	(No symbol) [0x00007FF704061052]
	(No symbol) [0x00007FF7040518A4]
	BaseThreadInitThunk [0x00007FFBE5D57344+20]
	RtlUserThreadStart [0x00007FFBE73626B1+33]

at alert Message: no such alert
  (Session info: chrome=124.0.6367.119)
Stacktrace:
	GetHandleVerifier [0x00007FF7040E1502+60802]
	(No symbol) [0x0

# 테스트코드는 여기 이하에 작성

In [11]:

for k, v in bloggers.items() :
    print(v.file_manager.get_texts())

AttributeError: 'File_manager' object has no attribute 'get_texts'